In [1]:
import warnings
warnings.filterwarnings('ignore')
import json
from pymongo import MongoClient, GEO2D

In [2]:
prescriptions_list = []
for line in open('prescriptions.jsonl', 'r'):
    prescriptions_list.append(json.loads(line))

In [3]:
myclient = MongoClient("mongodb://localhost:27017/")

#use database named "organisation"
mydb = myclient["HW2"]

mycol = mydb['hw2_prescriptions']

In [12]:
mycol.insert_many(prescriptions_list)

In [13]:
mydb.list_collection_names()

['hw2_zips', 'hw2_prescriptions']

In [14]:
list(mydb.hw2_prescriptions.find_one().keys())

['_id', 'cms_prescription_counts', 'provider_variables', 'npi']

In [5]:
mycol.find_one()

{'_id': ObjectId('6107af558851e7995ec536a2'),
 'cms_prescription_counts': {'DOXAZOSIN MESYLATE': 26,
  'MIDODRINE HCL': 12,
  'MEGESTROL ACETATE': 11,
  'BENAZEPRIL HCL': 11,
  'METOLAZONE': 73,
  'NOVOLOG': 12,
  'DIAZEPAM': 24,
  'HYDRALAZINE HCL': 50,
  'SENSIPAR': 94,
  'LABETALOL HCL': 28,
  'PREDNISONE': 40,
  'CALCITRIOL': 79,
  'HYDROCODONE-ACETAMINOPHEN': 64,
  'HYDROCHLOROTHIAZIDE': 59,
  'LOSARTAN-HYDROCHLOROTHIAZIDE': 14,
  'FENOFIBRATE': 14,
  'MINOXIDIL': 14,
  'MELOXICAM': 29,
  'ATENOLOL': 62,
  'CARISOPRODOL': 40,
  'GABAPENTIN': 35,
  'OMEPRAZOLE': 35,
  'KLOR-CON M10': 20,
  'LANTUS': 20,
  'AMLODIPINE BESYLATE': 175,
  'CARVEDILOL': 36,
  'LOSARTAN POTASSIUM': 41,
  'IRBESARTAN': 11,
  'NIFEDICAL XL': 32,
  'NIFEDIPINE ER': 51,
  'LEVOTHYROXINE SODIUM': 12,
  'POTASSIUM CHLORIDE': 30,
  'FUROSEMIDE': 162,
  'GLYBURIDE': 16,
  'CLONIDINE HCL': 43,
  'TEMAZEPAM': 41,
  'SPIRONOLACTONE': 50,
  'LOVASTATIN': 11,
  'LISINOPRIL': 44,
  'PANTOPRAZOLE SODIUM': 13,
  'CALCIU

## Q6 
Find the specialty of all doctors who have prescribed "HALOPERIDOL".

In [6]:
Q6_pipeline_hal = {'$match': {"cms_prescription_counts.HALOPERIDOL": {"$gt":0}}}
Q6_pipeline_speciality = { '$group' : { '_id' : "$provider_variables.specialty" } } 
# Q6_pipeline_hal = {"cms_prescription_counts.HALOPERIDOL":{"$exists":True}}

print_list = "The specialty of all doctors who have prescribed HALOPERIDOL include "
for _id in list(mycol.aggregate([Q6_pipeline_hal,Q6_pipeline_speciality])):
    print_list = print_list + _id['_id'] + ', '
print(print_list[:-2]+'.')

The specialty of all doctors who have prescribed HALOPERIDOL include Diagnostic Neuroimaging, Addiction (Substance Use Disorder), Adult Mental Health, Psych/Mental Health, Chronically Ill, Anatomic Pathology & Clinical Pathology, Psychiatric, Mental Health, Clinical Neurophysiology, Allergy & Immunology, Behavioral Neurology & Neuropsychiatry, Oral and Maxillofacial Surgery, Child & Adolescent Psychiatry, Gastroenterology, Forensic Psychiatry, Prescribing (Medical), Endocrinology, Diabetes & Metabolism, Neurology with Special Qualifications in Child Neurology, Pediatric Gastroenterology, Pediatrics, Clinical, Gynecologic Oncology, Emergency Medical Services, Primary Care, Psych/Mental Health, Geropsychiatric, Hematology, Public Health & General Preventive Medicine, Psych/Mental Health, Community, Mental Health (Including Community Mental Health Center), Professional, Critical Care Medicine, Pain Medicine, Interventional Cardiology, Nephrology, Sports Medicine, Hospice and Palliative Me

## Q7
Find the total number of doctors, separately for each region (in one query).

In [7]:
Q7_pipe_count =   {
     '$group': { 
        '_id': "$provider_variables.region", 
        'Number of Doctor': {'$sum': 1}
     }
  }

for _id in list(mycol.aggregate([Q7_pipe_count])):
    print('The number of Doctor in {} is {}.'.format(_id['_id'],_id['Number of Doctor'])) 

The number of Doctor in Midwest is 50077.
The number of Doctor in South is 80562.
The number of Doctor in Northeast is 59012.
The number of Doctor in West is 50279.


## Q8
Find the total amount of prescribed "ATORVASTATIN CALCIUM"

In [8]:
Q7_pipe_sum = {'$group': {'_id': None, 'total': {'$sum': '$cms_prescription_counts.ATORVASTATIN CALCIUM'}}}
print('The total amount of prescribed ATORVASTATIN CALCIUM is {}.'.format(list(mycol.aggregate([Q7_pipe_sum]))[0]['total']))

The total amount of prescribed ATORVASTATIN CALCIUM is 4534888.


## Q9
Find the drug that is prescribed by the most of doctors working in "non-urban" areas. (in terms of number of doctors who prescribed it, not the total amount of prescriptions).

In [47]:
Q9_match =  {"$match": {"provider_variables.settlement_type": "non-urban"}}
Q9_toarry =  { '$project': { 'prescribed': { '$objectToArray': "$cms_prescription_counts" } } }
Q9_unwind = { '$unwind': "$prescribed" }
Q9_count =  {'$group':{'_id':'$prescribed.k','number_of_doctors':{'$count':{}}}}
Q9_sort = {'$sort' : { 'number_of_doctors' : -1} }

print('The drug that is prescribed by the most of doctors working in non-urban areas is {}.'.format(list(mycol.aggregate([Q9_match,Q9_toarry,Q9_unwind,Q9_count,Q9_sort]))[0]['_id']))

The drug that is prescribed by the most of doctors working in non-urban areas is HYDROCODONE-ACETAMINOPHEN.


## Q10
Considering the region of US states (Query #4) and the region of each doctor, find the average number of doctors per capita in each of the four regions in US.